In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms.shortest_paths.weighted import dijkstra_path_length

In [2]:
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, adjacency_matrix, feature_matrix):
        x = torch.mm(adjacency_matrix, feature_matrix)  # Aggregate neighbor features
        x = torch.relu(self.fc1(x))
        x = torch.mm(adjacency_matrix, x)  # Aggregate neighbor features again
        x = self.fc2(x)
        return x


In [3]:
def optimal_pathfinding(adjacency_matrix, feature_matrix, source_node, target_node):
    num_nodes = feature_matrix.shape[0]
    input_dim = feature_matrix.shape[1]
    hidden_dim = 16
    output_dim = 1
    model = GNN(input_dim, hidden_dim, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()
    
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(adjacency_matrix, feature_matrix)
        target = torch.zeros((num_nodes, output_dim))
        target[target_node] = 1  # Set target node as 1, rest as 0
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Find the path with the highest GNN output from source to target
    current_node = source_node
    shortest_path = [current_node]
    max_iterations = num_nodes  # Set maximum number of iterations
    
    while current_node != target_node and max_iterations > 0:
        neighbors = torch.nonzero(adjacency_matrix[current_node]).squeeze()
        neighbor_scores = model(adjacency_matrix, feature_matrix)[neighbors]
        if len(neighbor_scores) == 0:
            break
        next_node = neighbors[torch.argmax(neighbor_scores)]
        current_node = next_node.item()
        shortest_path.append(current_node)
        max_iterations -= 1
    
    return shortest_path, model.fc1.weight.detach().numpy()

In [4]:
def calculate_accuracy(shortest_path, target_path):
    intersection = set(shortest_path).intersection(target_path)
    accuracy = len(intersection) / len(target_path)
    return accuracy

In [5]:
def dijkstra_shortest_path(adjacency_matrix, cost_matrix, source_node, target_node):
    graph = nx.from_numpy_array(cost_matrix, create_using=nx.DiGraph())
    length = dijkstra_path_length(graph, source=source_node, target=target_node)
    return length


In [6]:

import networkx as nx

# Example adjacency matrix and feature matrix (as previously defined)
adjacency_matrix = np.array([
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=np.float32)

In [7]:
cost_matrix = np.array([
    [0, 0, 0, 5399.60, 4713.72, 4841.50, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6920.39, 5754.89, 5069.28, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 7148.16, 7275.94, 6110.40, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 3959.53, 3472.11, 3561.55, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 5069.27, 4238.42, 3750.99, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 6356.11, 5348.16, 4517.31, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 12933.07, 9066.66, 7523.21],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 300000000000000, 13311.95, 9445.56],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 300000000000000, 300000000000000, 13690.33],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=np.float32)


In [8]:
source_node = 0
target_node = 11
dijkstra_length = dijkstra_shortest_path(adjacency_matrix, cost_matrix, source_node, target_node)
dijkstra_length

16882.340087890625